In [27]:
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj
import matplotlib.pyplot as plt

In [28]:
input_path = 'E:\\forked_canada_udp/data/inputs/'
output_path ='E:\\forked_canada_udp/data/outputs/'
city_name = "Vancouver"
all_typology = pd.read_csv(output_path+ str.lower(city_name)+'_final_output.csv', index_col = 0)

In [29]:
all_typology.groupby("typ_cat").size()

typ_cat
['ARE']           104
['ARG']             8
['AdvG']           10
['BE']             21
['EOG']             3
['LISD']           54
['OD']              9
['SAE', 'SGE']      5
['SAE']            50
['SMMI']          160
dtype: int64

### Typologies

In [30]:
all_typology.groupby("Typology").size()

Typology
Advanced Gentrification                    19
At Risk of Becoming Exclusive             104
At Risk of Gentrification                   8
Becoming Exclusive                         21
Early/Ongoing Gentrification                7
Low-Income/Susceptible to Displacement     46
Ongoing Displacement                        4
Stable Moderate/Mixed Income              160
Stable/Advanced Exclusive                  50
Super Gentrification and Exclusion          5
dtype: int64

### Findings: low-income households

In [26]:
# total number of low income households across the Vancouver CMA
li_hh_tot = all_typology["all_li_count_16"].sum()
li_hh_tot

368751.2460212461

In [29]:
# census tracts classified as EOG / ARG / LISD / OD
ARG_EOG_LISD_OD = (all_typology["Typology"] == "Early/Ongoing Gentrification") | (all_typology["Typology"] == "At Risk of Gentrification") | (all_typology["Typology"] == "Low-Income/Susceptible to Displacement") | (all_typology["Typology"] == "Ongoing Displacement") 

In [32]:
# total number of low income households living in an ARG or EOG census tract
li_ARG_EOG_LISD_OD = all_typology[ARG_EOG_LISD_OD]["all_li_count_16"].sum()
li_ARG_EOG_LISD_OD

100026.42228142229

In [33]:
# percentage of low income households living in an ARG or EOG tract
pct_li_ARG_or_EOG = li_ARG_EOG_LISD_OD / li_hh_tot
pct_li_ARG_or_EOG

0.27125717773346625

In [34]:
# geographic distribution of low income households in EOG or ARG tracts
all_typology[ARG_EOG_LISD_OD == 1].groupby("Region.Name").agg('sum')["all_li_count_16"].sort_values(ascending = False) / li_ARG_EOG_LISD_OD

Region.Name
Vancouver              0.368164
Richmond               0.135369
Surrey                 0.105596
Burnaby                0.096362
Coquitlam              0.069099
Langley                0.050581
Greater Vancouver A    0.034483
Maple Ridge            0.034106
North Vancouver        0.030629
White Rock             0.026995
New Westminster        0.026298
Capilano 5             0.022317
Name: all_li_count_16, dtype: float64

In [21]:
# distribution of low income households according to typology
(all_typology.groupby("Typology").agg('sum')["all_li_count_16"].sort_values(ascending = False) / li_hh_tot).sort_values(ascending = False)

Typology
Stable Moderate/Mixed Income              0.360276
At Risk of Becoming Exclusive             0.206639
Low-Income/Susceptible to Displacement    0.193576
Advanced Gentrification                   0.061792
Stable/Advanced Exclusive                 0.051374
Becoming Exclusive                        0.045650
Early/Ongoing Gentrification              0.031918
At Risk of Gentrification                 0.027675
Ongoing Displacement                      0.018088
Super Gentrification and Exclusion        0.003012
Name: all_li_count_16, dtype: float64

In [36]:
proportions = all_typology.groupby(["Typology", "Region.Name"]).agg('sum')["hh_16"] / all_typology.groupby(["Region.Name", ""["hh_16"].sum()).sort_values(ascending = False)
proportions

Typology
Stable Moderate/Mixed Income              0.371894
At Risk of Becoming Exclusive             0.244330
Low-Income/Susceptible to Displacement    0.136632
Stable/Advanced Exclusive                 0.090096
Advanced Gentrification                   0.049920
Becoming Exclusive                        0.048992
Early/Ongoing Gentrification              0.020283
At Risk of Gentrification                 0.018832
Ongoing Displacement                      0.013124
Super Gentrification and Exclusion        0.005898
Name: hh_16, dtype: float64

Typology
Advanced Gentrification                   0.044811
At Risk of Becoming Exclusive             0.245283
At Risk of Gentrification                 0.018868
Becoming Exclusive                        0.049528
Early/Ongoing Gentrification              0.016509
Low-Income/Susceptible to Displacement    0.108491
Ongoing Displacement                      0.009434
Stable Moderate/Mixed Income              0.377358
Stable/Advanced Exclusive                 0.117925
Super Gentrification and Exclusion        0.011792
dtype: float64

In [130]:
proportions.values[1] + proportions.values[3] + proportions.values[8] + proportions.values[9]

0.3985849056603774

In [37]:
gentrification = all_typology[(all_typology["Typology"] == "Advanced Gentrification") |
                             (all_typology["Typology"] == "At Risk of Gentrification") |
                             (all_typology["Typology"] == "Early/Ongoing Gentrification")]

In [39]:
(gentrification.groupby("Region.Name").agg('sum')["hh_16"] / gentrification["hh_16"].sum()).sort_values(ascending = False)

Region.Name
Vancouver              0.338021
Burnaby                0.215776
New Westminster        0.163475
Surrey                 0.134600
Port Moody             0.051404
Maple Ridge            0.028476
Greater Vancouver A    0.026531
White Rock             0.022529
Richmond               0.019188
Name: hh_16, dtype: float64

In [307]:
gentrification.groupby("Region.Name").size().sum() / all_typology.groupby("Region.Name").size().sum()

0.08018867924528301

In [40]:
exclusive = all_typology[(all_typology["Typology"] == "Becoming Exclusive") |
                             (all_typology["Typology"] == "At Risk of Becoming Exclusive") |
                             (all_typology["Typology"] == "Stable/Advanced Exclusive") |
                             (all_typology["Typology"] == "Super Gentrification and Exclusion")
                        ]

In [309]:
exclusive.groupby("Region.Name").agg('sum')["hh_16"] / all_typology.groupby("Region.Name").size().sum()

0.42452830188679247

In [41]:
(exclusive.groupby("Region.Name").agg('sum')["hh_16"] / exclusive["hh_16"].sum()).sort_values(ascending = False)

Region.Name
Surrey             0.279591
Vancouver          0.174393
Langley            0.081481
North Vancouver    0.074379
Delta              0.067162
Port Coquitlam     0.062024
Burnaby            0.058637
Coquitlam          0.050602
Maple Ridge        0.046733
Richmond           0.036408
Port Moody         0.025029
New Westminster    0.016127
Pitt Meadows       0.010327
White Rock         0.009441
West Vancouver     0.007667
Name: hh_16, dtype: float64